In [3]:
import pandas as pd
import yfinance as yf

In [5]:
def calcular_rentabilidade_e_analise(dataframe):
    # Calcula a rentabilidade com base no pct_change()
    dataframe['Rentabilidade'] = dataframe['Close'].pct_change()

    # Substituir valores NaN por 0 na coluna 'Rentabilidade'
    dataframe['Rentabilidade'].fillna(0, inplace=True)
    
    analise_movimentos = []
    inicio_movimento = None
    sequencia_atual = 0
    max_sequencia = 0
    rentabilidade_acc = 1
    rentabilidade_ant = 1

    # Itera sobre cada linha do dataframe
    for index, row in dataframe.iterrows():
        # Detecta início de uma sequência
        if inicio_movimento is None:
            inicio_movimento = index
            rentabilidade_acc = 1 + row['Rentabilidade']
        
        # Detecta sequência de alta
        if row['Rentabilidade'] < 0 and rentabilidade_ant > 0:
            fim_movimento = index - pd.DateOffset(1) 
            analise_movimentos.append({
                'Sequencia': sequencia_atual,
                'Tipo': 'Alta',
                'Inicio': inicio_movimento,
                'Fim': fim_movimento,
                'Rentabilidade_Acumulada': (rentabilidade_acc - 1) * 100
            })
            inicio_movimento = index
            sequencia_atual = 1
            max_sequencia = max(max_sequencia, sequencia_atual)
            rentabilidade_acc = 1 + row['Rentabilidade']
        # Detecta sequência de baixa
        elif row['Rentabilidade'] > 0 and rentabilidade_ant < 0:
            fim_movimento = index - pd.DateOffset(1) 
            analise_movimentos.append({
                'Sequencia': sequencia_atual,
                'Tipo': 'Baixa',
                'Inicio': inicio_movimento,
                'Fim': fim_movimento,
                'Rentabilidade_Acumulada': (rentabilidade_acc - 1) * 100
            })
            inicio_movimento = index
            sequencia_atual = 1
            max_sequencia = max(max_sequencia, sequencia_atual)
            rentabilidade_acc = 1 + row['Rentabilidade']
        else:
            sequencia_atual += 1
            max_sequencia = max(max_sequencia, sequencia_atual)
            rentabilidade_acc *= 1 + row['Rentabilidade']
        
        rentabilidade_ant = row['Rentabilidade']
    
    # Inserção do último registro
    fim_movimento = index
    analise_movimentos.append({
        'Sequencia': sequencia_atual,
        'Tipo': 'Baixa' if rentabilidade_acc < 1 else 'Alta',
        'Inicio': inicio_movimento,
        'Fim': fim_movimento,
        'Rentabilidade_Acumulada': (rentabilidade_acc - 1) * 100
    })
    
    analise_movimentos_df = pd.DataFrame(analise_movimentos)
    
    return dataframe, analise_movimentos_df

In [4]:
# Baixa os dados da ação desejada
df_weekly = yf.download('^BVSP', interval='1wk')
df_daily = yf.download('^BVSP', interval='1d')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [6]:
df_daily, analise_movimentos_daily = calcular_rentabilidade_e_analise(df_daily)

In [7]:
analise_movimentos_daily[analise_movimentos_daily['Sequencia']== 12]

,Sequencia,Tipo,Inicio,Fim,Rentabilidade_Acumulada
5,12,Alta,1993-05-11,1993-05-26,38.888885
442,12,Alta,1997-05-15,1997-06-02,10.483322
2974,12,Alta,2017-12-20,2018-01-08,9.217116


In [9]:
# Supondo que 'data' seja a coluna de datas no DataFrame
start_date = '2017-12-20'
end_date = '2018-01-08'

# Filtrar o DataFrame pelo intervalo de datas
df_daily.loc[(df_daily.index >= start_date) & (df_daily.index <= end_date)]['Rentabilidade']

Date
2017-12-20    0.009452
2017-12-21    0.024071
2017-12-22    0.000719
2017-12-26    0.006663
2017-12-27    0.005087
2017-12-28    0.004325
2017-12-29    0.000000
2018-01-02    0.019489
2018-01-03    0.001335
2018-01-04    0.008360
2018-01-05    0.005391
2018-01-08    0.003895
Name: Rentabilidade, dtype: float64